In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [ ]:
class Team:
    def __init__(self, name, units):
        self.name = name
        self.units = units

In [ ]:
class Unit:
    def __init__(self, name, initiative, strength, armour, health):
        self.name = name
        self.initiative = initiative
        self.strength = strength
        self.armour = armour
        self.current_health = health
        self.health = health

    def reset(self):
        self.current_health = self.health

In [ ]:
def faceoff_phase(team_a, team_b):
    roll_a = 0
    roll_b = 0

    for unit in team_a.units:
        roll = random.randint(1, 6) + unit.initiative
        if roll > roll_a:
            roll_a = roll

    for unit in team_b.units:
        roll = random.randint(1, 6) + unit.initiative
        if roll > roll_b:
            roll_b = roll

    # Resolve ties with roll-offs
    if roll_a == roll_b:
        roll = random.randint(1, 6)
        if roll <= 3:
            roll_a = 6
            roll_b = 1
        else:
            roll_a = 1
            roll_b = 6

    #Result
    if roll_a > roll_b:
        return True
    else:
        return False

In [ ]:
def strike_phase(team_a, team_b, winner):
    if winner == False:
        return 1.

    for attacker in team_a.units:
        # Filter out defeated units
        alive_defenders = [unit for unit in team_b.units if unit.current_health > 0]
        if not alive_defenders:
            break  # All defenders are down

        #Grab first defender
        defender = alive_defenders[0]

        #Just chek incase the roll is impossible
        if attacker.strength + 6 <= defender.armour:
            _rand = 10
            roll = random.randint(1, _rand)
            if roll == _rand:
                defender.current_health -= 1
        else:
            #Do the roll
            roll = random.randint(1, 6) + attacker.strength
            if roll > defender.armour:
                defender.current_health -= 1

    # Calculate health ratio of losing team
    current_health = sum(unit.current_health for unit in team_b.units)
    total_health = sum(unit.health for unit in team_b.units)
    return current_health / total_health

In [ ]:
# Monte Carlo simulation
def monte_carlo_simulation(initiative=5, strength=5, armour=10, health=1, trials=1000):
    team1_unit = Unit(name="Red Warrior", initiative=initiative, strength=strength, armour=5, health=health)
    team2_unit = Unit(name="Blue Knight", initiative=5, strength=5, armour=armour, health=1)

    team1 = Team("Team 1", [team1_unit])
    team2 = Team("Team 2", [team2_unit])

    zero_count = 0
    for _ in range(trials):
        # Reset health
        for unit in team1.units:
            unit.reset()
        for unit in team2.units:
            unit.reset()

        winner = faceoff_phase(team1, team2)
        result = strike_phase(team1, team2, winner)

        if result == 0:
            zero_count += 1

    return (zero_count / trials) * 100

In [ ]:
# Interactive plotting function
def plot_heatmap(value):
    # Set up grid parameters
    initiative_range = range(1, 11)
    strength_range = range(1, 11)
    heatmap = np.zeros((len(strength_range), len(initiative_range)))

    # Run simulations across the grid
    for i, strength in enumerate(strength_range):
        for j, initiative in enumerate(initiative_range):
            heatmap[i, j] = monte_carlo_simulation(initiative=initiative, strength=strength, armour=value, trials=100)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, origin='lower', aspect='auto', cmap='viridis', extent=[1, 11, 1, 11], vmin=0, vmax=100)
    plt.colorbar(label='Percentage of times result is 0')
    plt.xlabel('Initiative')
    plt.ylabel('Strength')
    plt.show()

# Create the interactive slider
interact(plot_heatmap, value=IntSlider(min=1, max=20, step=1, value=5, description='Initiative'));

In [ ]:
# Interactive plotting function
def plot_heatmap(value):
    # Set up grid parameters
    armour_range = range(1, 11)
    strength_range = range(1, 11)
    heatmap = np.zeros((len(strength_range), len(armour_range)))

    # Run simulations across the grid
    for i, strength in enumerate(strength_range):
        for j, armour in enumerate(armour_range):
            heatmap[i, j] = monte_carlo_simulation(initiative=value, strength=strength, armour=armour, trials=100)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, origin='lower', aspect='auto', cmap='viridis', extent=[1, 11, 1, 11], vmin=0, vmax=100)
    plt.colorbar(label='Percentage of times result is 0')
    plt.xlabel('Armour')
    plt.ylabel('Strength')
    plt.show()

# Create the interactive slider
interact(plot_heatmap, value=IntSlider(min=1, max=10, step=1, value=5, description='Initiative'));